<a href="https://colab.research.google.com/github/ayush-shirode/ML_Models/blob/main/coco_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !unzip -q "/content/drive/MyDrive/COCO Dataset/annotations_trainval2017.zip" -d "/content/drive/MyDrive/coco"

In [ ]:
# !unzip -n "/content/drive/MyDrive/COCO Dataset/train2017.zip" -d "/content/drive/MyDrive/coco"

In [ ]:
# !unzip -q "/content/drive/MyDrive/COCO Dataset/test2017.zip" -d "/content/drive/MyDrive/coco"

In [ ]:
# !unzip -q "/content/drive/MyDrive/COCO Dataset/val2017.zip" -d "/content/drive/MyDrive/coco"

In [ ]:
!ls /content/drive/MyDrive/coco

annotations  coco_1.yaml  coco.yaml  images  labels


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.1 MB/s eta 0:00:00


In [ ]:
import os

print(os.path.exists("/content/drive"))


True


In [ ]:
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
# !mkdir -p /content/drive/MyDrive/coco/images
# !mkdir -p /content/drive/MyDrive/coco/labels

In [ ]:
# !mv /content/drive/MyDrive/coco/train2017 /content/drive/MyDrive/coco/images/
# !mv /content/drive/MyDrive/coco/val2017 /content/drive/MyDrive/coco/images/
# !mv /content/drive/MyDrive/coco/test2017 /content/drive/MyDrive/coco/images/

In [ ]:
rm -rf /content/coco/labels

In [ ]:
import json
import os
from tqdm import tqdm

def convert_coco_to_yolo(coco_json_path, output_dir, images_dir):
    # Load COCO JSON
    with open(coco_json_path, 'r') as f:
        coco = json.load(f)

    # Create category ID → continuous class index mapper
    categories = coco['categories']
    cat2idx = {cat['id']: idx for idx, cat in enumerate(categories)}

    # Collect all bbox annotations by image_id
    img2ann = {img['id']: [] for img in coco['images']}
    for ann in coco['annotations']:
        img2ann[ann['image_id']].append(ann)

    os.makedirs(output_dir, exist_ok=True)

    # Convert each image's annotations
    for img in tqdm(coco['images'], desc=f"Converting {os.path.basename(coco_json_path)}"):
        img_id = img['id']
        file_name = img['file_name']
        w, h = img['width'], img['height']

        label_path = os.path.join(output_dir, file_name.replace(".jpg", ".txt"))

        with open(label_path, "w") as f:
            for ann in img2ann[img_id]:
                cat_id = ann['category_id']
                cls = cat2idx[cat_id]

                # COCO bbox: [x_min, y_min, width, height]
                x_min, y_min, bw, bh = ann['bbox']

                # Convert to YOLO format
                x_center = (x_min + bw / 2) / w
                y_center = (y_min + bh / 2) / h
                bw /= w
                bh /= h

                # Write label
                f.write(f"{cls} {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")

    print(f"✔ Done! Labels saved to {output_dir}")


# ================================
# RUN CONVERSION FOR TRAIN + VAL
# ================================
base = "/content/coco"

convert_coco_to_yolo(
    coco_json_path=f"/content/drive/MyDrive/coco/annotations/instances_train2017.json",
    output_dir=f"{base}/labels/train2017",
    images_dir=f"{base}/train2017"
)

convert_coco_to_yolo(
    coco_json_path=f"/content/drive/MyDrive/coco/annotations/instances_val2017.json",
    output_dir=f"{base}/labels/val2017",
    images_dir=f"{base}/val2017"
)


Converting instances_train2017.json: 100%|██████████| 118287/118287 [00:09<00:00, 11956.95it/s]


✔ Done! Labels saved to /content/coco/labels/train2017


Converting instances_val2017.json: 100%|██████████| 5000/5000 [00:00<00:00, 15580.89it/s]

✔ Done! Labels saved to /content/coco/labels/val2017


In [ ]:
# !rsync -a --info=progress2 "/content/drive/MyDrive/coco/" "/content/coco/"

In [ ]:
!mkdir -p /content/coco/images
!unzip -q "/content/drive/MyDrive/COCO Dataset/train2017.zip" -d "/content/coco/images/"
!unzip -q "/content/drive/MyDrive/COCO Dataset/val2017.zip"   -d "/content/coco/images/"
!unzip -q "/content/drive/MyDrive/COCO Dataset/test2017.zip"  -d "/content/coco/images/"

In [ ]:
!cp -r "/content/drive/MyDrive/coco/labels/train2017" "/content/coco/labels/train"
!cp -r "/content/drive/MyDrive/coco/labels/val2017"   "/content/coco/labels/val"
# !cp -r "/content/drive/MyDrive/coco/labels/test2017"  "/content/coco/labels/test"


^C
^C


In [ ]:
model.train(
    data="/content/drive/MyDrive/coco/coco_1.yaml",
    imgsz=640,
    epochs=10,
    batch=16,
    device=0
)


Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/coco/coco_1.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=Tru